In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm #progress bar 
from collections import Counter
import json

#Reading of Data

In [0]:
def read_data():
  specs = pd.read_csv("/content/drive/My Drive/introml/data-science-bowl-2019/specs.csv")
  test = pd.read_csv("/content/drive/My Drive/introml/data-science-bowl-2019/test.csv")
  train = pd.read_csv("/content/drive/My Drive/introml/data-science-bowl-2019/train.csv")
  train_labels = pd.read_csv("/content/drive/My Drive/introml/data-science-bowl-2019/train_labels.csv")

  return train, test, train_labels, specs

#Preprocessing

In [0]:
#dropping 
# 1. title --> welcome to lost lagoon is a start screen (world == None) (can drop)
# 2. some take the assessment but are not in train_labels 
# 3. some didn’t take assessment
# 4. Event data (its already out into individual columns) 

def remove_people_w_no_assessment(train, test, train_labels):
  #drop rows that are not found inside train_label data --> never take assessment
  assessment_data_id = list(train_labels['installation_id'].unique())
  train = train[train['installation_id'].isin(assessment_data_id)]

  return train, test
  # train, test = removal_of_redundant_data(train, test)


def removal_of_redundant_data(train, test):
  #drop world == NONE
  train = train[train['world'] != 'NONE']
  test = test[test['world'] != 'NONE']
  return train, test

In [0]:
train, test, train_labels, specs = read_data()

train, test = remove_people_w_no_assessment(train, test, train_labels)
train, test = removal_of_redundant_data(train, test)

In [0]:
train.head()

,event_id,game_session,timestamp,event_data,installation_id,event_count,event_code,game_time,title,type,world
1539,27253bdc,4b57c9a59474a1b9,2019-08-06T04:57:45.301Z,"{""event_code"": 2000, ""event_count"": 1}",0006a69f,1,2000,0,Magma Peak - Level 1,Clip,MAGMAPEAK
1540,77261ab5,2b9d5af79bcdb79f,2019-08-06T04:58:14.538Z,"{""version"":""1.0"",""event_count"":1,""game_time"":0...",0006a69f,1,2000,0,Sandcastle Builder (Activity),Activity,MAGMAPEAK
1541,b2dba42b,2b9d5af79bcdb79f,2019-08-06T04:58:14.615Z,"{""description"":""Let's build a sandcastle! Firs...",0006a69f,2,3010,29,Sandcastle Builder (Activity),Activity,MAGMAPEAK
1542,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:16.680Z,"{""coordinates"":{""x"":273,""y"":650,""stage_width"":...",0006a69f,3,4070,2137,Sandcastle Builder (Activity),Activity,MAGMAPEAK
1543,1325467d,2b9d5af79bcdb79f,2019-08-06T04:58:18.474Z,"{""coordinates"":{""x"":863,""y"":237,""stage_width"":...",0006a69f,4,4070,3937,Sandcastle Builder (Activity),Activity,MAGMAPEAK


#Feature engineering

##Encoding the data 
- Encode categorical data into numerical ones
- Store dictionaries of index and actual labels for future use

In [0]:
def encode_titles(train, test, train_labels):
  train['edited_title'] = train['title'].astype(str).str.replace(' ', '')
  train['title_event_code'] = train['edited_title'].astype(str) + '_' + train['event_code'].astype(str)
  test['title_event_code'] = test['title'].astype(str) + '_' +  test['event_code'].astype(str)

  train['type_world'] = train['type'] + '_' + train['world']
  test['type_world'] = test['type'] + '_' + test['world']

  #make a list of all the unique values of columns that are not numbers
  list_of_all_titles = train['title'].unique()
  list_of_all_title_event_code = train['title_event_code'].unique()
  list_of_all_type_world = train['type_world'].unique()
  list_of_all_event_id = train['event_id'].unique()
  list_of_all_event_code = train['event_code'].unique()
  list_of_all_assessment_title = train[train['type'] == 'Assessment']['title'].unique()
  list_of_all_world = train['world'].unique()

  list_of_all_installation_id = list(set(train['installation_id'].unique()).union(test['installation_id'].unique()))
  list_of_all_game_session = list(set(train['game_session'].unique()).union(test['game_session'].unique()))

  #making a dictionary of all the list with keys for mapping later:
  dict_of_all_titles = dict(zip(list_of_all_titles, range(len(list_of_all_titles))))
  dict_of_all_labels = dict(zip(range(len(list_of_all_titles)), list_of_all_titles))
  # dict_of_all_type_worlds = dict(zip(list_of_all_type_world, range(len(list_of_all_type_world))))
  dict_of_all_world_labels = dict(zip(range(len(list_of_all_world)), list_of_all_world))
  dict_of_all_worlds = dict(zip(list_of_all_world, range(len(list_of_all_world))))

  dict_of_all_installation_ids = dict(zip(list_of_all_installation_id, range(len(list_of_all_installation_id))))
  dict_of_all_game_session = dict(zip(list_of_all_game_session, range(len(list_of_all_game_session))))


  #replace text with numbers from the dict above:
  train['title'] = train['title'].map(dict_of_all_titles)
  test['title'] = test['title'].map(dict_of_all_titles)

  train['world'] = train['world'].map(dict_of_all_worlds)
  test['world'] = test['world'].map(dict_of_all_worlds)

  train_labels['title'] = train_labels['title'].map(dict_of_all_worlds)

  #get the dict of win code 4100 but for bird measurer is 4110
  win_code = dict(zip(dict_of_all_titles.values(), (4100 * np.ones(len(dict_of_all_titles))).astype('int')))
  win_code[dict_of_all_titles['Bird Measurer (Assessment)']] = 4110

  #converting to datetime
  train['timestamp'] = pd.to_datetime(train['timestamp'])
  test['timestamp'] = pd.to_datetime(test['timestamp'])

  #putting all the list in to a dictionary
  event_data = {}
  event_data['win_code'] = win_code
  event_data['list_of_titles'] = list_of_all_titles
  event_data['list_of_all_title_event_code'] = list_of_all_title_event_code
  event_data['list_of_all_type_world'] = list_of_all_type_world
  event_data['list_of_all_event_id'] = list_of_all_event_id
  event_data['list_of_all_event_code'] = list_of_all_event_code
  event_data['list_of_all_assessment_title'] = list_of_all_assessment_title
  event_data['list_of_all_world'] = list_of_all_world
  event_data['dict_of_all_titles'] = dict_of_all_titles
  event_data['dict_of_all_worlds'] = dict_of_all_worlds
  event_data['dict_of_all_labels'] = dict_of_all_labels
  event_data['dict_of_all_world_labels'] = dict_of_all_world_labels
  event_data['dict_of_all_installation_ids'] = dict_of_all_installation_ids
  event_data['dict_of_all_game_session'] = dict_of_all_game_session
 
  #dropping of useless columns:
  train.drop(['edited_title'], axis=1, inplace=True)

  return train, test, event_data

##Function to engineer features for each user
- Create features for Game, Activity and Assessment

In [0]:
#input user is a groupby of a installation ID with all the info of that installation inside
def get_data_for_each_user(user, event_data, train_labels, test=False):
  last_assessment = {}
  last_activity = 0

  #dictionary to store the number of times the user has done for each type of activity
  count_of_users_activities = {'Clip':0, 'Activity':0, 'Assessment':0, 'Game':0}

  # dictionary to store the count of each title
  count_of_title = {title: 0 for title in event_data['list_of_titles']}
  count_of_event_id = {eid: 0 for eid in event_data['list_of_all_event_id']}
  count_of_event_code = {code: 0 for code in event_data['list_of_all_event_code']}
  count_of_world = {world: 0 for world in event_data['list_of_all_world']}

  #dictionary to store the total game time spent for each user 
  game_time_dict = {'Game_gametime': 0,'Activity_gametime': 0, 'Assessment_gametime': 0}

  #dictionary to store accuracy group
  count_of_accuracy_group = {0:0, 1:0, 2:0, 3:0}

  #variables to keep track 
  activity_type_event_count = 0
  activity_durations = []
  last_activity_features = {}

  game_type_event_count = 0
  total_game_misses = 0
  game_durations = []
  last_game_features = {}
  mean_game_round = 0

  assessment_type_event_count = 0
  assessment_durations = []
  last_assessment_features = {}
  indi_assessment_sum = {
      'correct_Mushroom Sorter (Assessment)' : 0,
      'wrong_Mushroom Sorter (Assessment)' : 0,
      'correct_Bird Measurer (Assessment)' : 0,
      'wrong_Bird Measurer (Assessment)' : 0,
      'correct_Cart Balancer (Assessment)' : 0,
      'wrong_Cart Balancer (Assessment)' : 0,
      'correct_Chest Sorter (Assessment)' : 0,
      'wrong_Chest Sorter (Assessment)' : 0,
      'correct_Cauldron Filler (Assessment)' : 0,
      'wrong_Cauldron Filler (Assessment)' : 0
  }

  #dictionary of the number of corrects for each step of each assessment
  #cart balancer is not included here because the evernt data of event_code 4020 for cart balancer assesment do not have any data 
  #that tells us it is the correct move. but for other assessments, event_code 4020 will tell us if the user have made a correct or 
  #wrong move
  assessment_4020_dict= {'Cauldron Filler (Assessment)_4020_accuracy': 0,
                                  'Mushroom Sorter (Assessment)_4020_accuracy': 0,
                                  'Bird Measurer (Assessment)_4020_accuracy': 0,
                                  'Chest Sorter (Assessment)_4020_accuracy': 0}

  all_assessments = []

  for index, game_session in user.groupby('game_session'):
    count_of_title = count_occurrence(game_session, count_of_title, 'title')
    count_of_event_id = count_occurrence(game_session, count_of_event_id, 'event_id')
    count_of_event_code = count_occurrence(game_session, count_of_event_code, 'event_code')
    count_of_world = count_occurrence(game_session, count_of_world, 'world')
    # count_of_users_activities = count_occurrence(game_session, count_of_users_activities, 'type')

    #extract main information from the session
    game_session_id = game_session['game_session'].iloc[0]
    session_type = game_session['type'].iloc[0]
    session_title = game_session['title'].iloc[0]
    # print(session_title)
    # input()
    session_title_text = event_data["dict_of_all_labels"][session_title]
    session_world = game_session['world'].iloc[0]
    session_timestamp = game_session['timestamp'].iloc[0]


    if last_activity != session_type:
      count_of_users_activities[session_type] += 1
      last_activity = session_type

    #based on the session type, get the certain data out
    #eg. number of correct, duration

    ##### Activity Features #####
    if session_type == 'Activity':
      #take the last count for activity 
      # activity_type_event_count += game_session['event_count'].iloc[-1]
      #get the duration for that activity --> last activity move - start time of the activity
      activity_durations.append((game_session.iloc[-1,2] - game_session.iloc[0,2]).seconds)
      #get the features of the last activity the person did in the game session
      last_activity_features = {
          'last_activity_session_title' : session_title,
          'last_activity_timestamp': session_timestamp,
          'last_activity_world' : session_world,
          'last_activity_event_count': game_session['event_count'].values[-1],
          'last_activity_unique_number_of_event_count': game_session['event_count'].nunique()
    }
    ##### Game Features #####
    if session_type == 'Game':
      # get the number of games played so far
      # game_type_event_count += game_session['event_count'].iloc[-1] ^^^^^^^^^^
      # get the duration for the games played
      game_durations.append((game_session.iloc[-1, 2] - game_session.iloc[0, 2]).seconds)


      ## Special features for games ##
      # Number of rounds would indicate increased practice and training of the specific skill for the assessment
      try: # add to mean_game_round only if round is found in the event data
        game_round = json.loads(game_session['event_data'].iloc[-1])["round"] 
        mean_game_round = (mean_game_round + game_round) / 2.0
      except:
        pass

      #filter all event_data that are of event code 2030 to count the misses each user made 
      games_with_2030 = game_session[game_session['event_code'] == 2030]
      misses_count = count_miss(games_with_2030)

      total_game_misses += misses_count

      game_session_correct, game_session_wrong = count_num_correct(game_session)

      #get the features of the last game the person did in the game session
      last_game_features = {
          'last_game_session_title' : session_title,
          'last_game_timestamp': session_timestamp,
          'last_game_world' : session_world,
          'last_game_event_count': game_session['event_count'].values[-1],
          'last_game_unique_number_of_event_count': game_session['event_count'].nunique()
      }

    ##### Assessment Features #####
    if (session_type == "Assessment") & (test or len(game_session) > 1):
      
      #search for all the attempts of that assessment inside the session
      every_attempt = game_session.query(f'event_code == {event_data["win_code"][session_title]}')

      #based on the query, count the number of correct and wrong attempts
      correct_attempts = every_attempt['event_data'].str.contains('true').sum()
      wrong_attempts = every_attempt['event_data'].str.contains('false').sum()

      # calculating the number of correct and wrong attempts for each individual assessment type
      correct_session_title = 'correct_' + session_title_text
      wrong_session_title = 'wrong_' + session_title_text
      if correct_session_title in indi_assessment_sum:
        indi_assessment_sum[correct_session_title] += correct_attempts
        indi_assessment_sum[wrong_session_title] += wrong_attempts 
      else:
        indi_assessment_sum[correct_session_title] = correct_attempts
        indi_assessment_sum[wrong_session_title] = wrong_attempts

      # accuracy derived from code 4020: indication if state is correct or not
      assessment_4020_dict = assessment_4020_acc(game_session, assessment_4020_dict, event_data)

      # Features to be returned whenever an assessment is encountered
      final_features = count_of_users_activities.copy()
      # final_features['session_title_text'] = session_title_text
      final_features['session_title'] = session_title
      final_features.update(count_of_title.copy())
      
      final_features.update(count_of_accuracy_group.copy())
      final_features.update(count_of_event_id.copy())
      final_features.update(count_of_event_code.copy())
      final_features.update(count_of_world.copy())
      final_features.update(assessment_4020_dict.copy())
      final_features.update(indi_assessment_sum.copy())

      final_features['game_session'] = event_data['dict_of_all_game_session'][game_session_id]
      # final_features['activity_type_event_count'] = activity_type_event_count
      # final_features['game_type_event_count'] = game_type_event_count ^^^^^^^^^^^^^^^^^
      final_features['total_game_misses'] = total_game_misses
      final_features['mean_game_round'] = mean_game_round
      # final_features['assessment_type_event_count'] = assessment_type_event_count

      final_features['installation_id'] = event_data['dict_of_all_installation_ids'][game_session['installation_id'].iloc[-1]]
      
      if last_activity_features != {}:
        final_features.update(last_activity_features.copy())
      else:
        final_features['last_activity_session_title'] = np.nan
        final_features['last_activity_timestamp'] =  np.nan
        final_features['last_activity_world'] = np.nan
        final_features['last_activity_event_count'] = np.nan
        final_features['last_activity_unique_number_of_event_count'] = np.nan

      if last_game_features != {}:
        final_features.update(last_game_features.copy())
      else:
        final_features['last_game_session_title'] = np.nan
        final_features['last_game_timestamp'] =  np.nan
        final_features['last_game_world'] = np.nan
        final_features['last_game_event_count'] = np.nan
        final_features['last_game_unique_number_of_event_count'] = np.nan

      if activity_durations != []:
        final_features['activity_durations_mean'] = np.mean(activity_durations)
        game_time_dict['Activity_gametime'] = np.sum(activity_durations)
      else:
        final_features['activity_durations_mean'] = np.nan

      if game_durations != []:
        final_features['game_durations_mean'] = np.mean(game_durations)
        game_time_dict['Game_gametime'] = np.sum(game_durations)
      else:
        final_features['game_durations_mean'] = np.nan

      if assessment_durations != []:
        final_features['assessment_durations_mean'] = np.mean(assessment_durations)
        game_time_dict['Assessment_gametime'] = np.sum(assessment_durations)
      else:
        final_features['assessment_durations_mean'] = np.nan

      assessment_type_event_count += game_session['event_count'].iloc[-1]
      assessment_durations.append((game_session.iloc[-1,2] - game_session.iloc[0,2]).seconds)

      final_features.update(game_time_dict.copy())

      ## Special features for Assessments ##

      # obtaining the ground truth label of accuracy_group from train_labels. if game_session is not found in train_labels, calculate accuracy according to competition rules
      current_game_session_id = game_session['game_session'].iloc[0]
      current_train_label = train_labels[train_labels['game_session'] == current_game_session_id]

      if len(current_train_label) == 0:
        accuracy = correct_attempts / (correct_attempts + wrong_attempts) if (correct_attempts + wrong_attempts) != 0 else 0

        if accuracy == 0:
            final_features['accuracy_group'] = 0
        elif accuracy == 1:
            final_features['accuracy_group'] = 3
        elif accuracy == 0.5:
            final_features['accuracy_group'] = 2
        else:
            final_features['accuracy_group'] = 1

      else:
        final_features['accuracy_group'] = current_train_label['accuracy_group'].iloc[0]


      if last_assessment_features != {}:
        final_features.update(last_assessment_features.copy())
      else:
        final_features['last_assessment_session_title'] = np.nan
        final_features['last_assessment_timestamp'] =  np.nan
        final_features['last_assessment_world'] = np.nan
        final_features['last_assessment_event_count'] = np.nan
        final_features['last_assessment_unique_number_of_event_count'] = np.nan
        final_features['last_assessment_accuracy_group'] = np.nan


      last_assessment_features = {
        'last_assessment_session_title' : session_title,
        'last_assessment_timestamp': session_timestamp,
        'last_assessment_world' : session_world,
        'last_assessment_event_count': game_session['event_count'].values[-1],
        'last_assessment_unique_number_of_event_count': game_session['event_count'].nunique(),
        'last_assessment_accuracy_group': final_features['accuracy_group']
      }


      count_of_accuracy_group[final_features['accuracy_group']] += 1
      final_features.update(count_of_accuracy_group)

      mean_accuracy_group = final_features['accuracy_group'] / sum(count_of_accuracy_group.values())

    
      if test:
        last_assessment = final_features.copy()

      if correct_attempts + wrong_attempts > 0:
        all_assessments.append(final_features)
  
  if test:
    return last_assessment
  
  return all_assessments

## Helper functions for the engineering of features
- count_occurances --> count the title and world for each session
- count_miss --> count the number of misses in the event code 
- count_number_correct --> count the number of correct moves the user made in the event code 
- assessment_4020_acc --> count the number of corrects and incorrects the user too in this assessment

In [0]:
# helper functions - overall
def count_occurrence(game_session, count_dict, column):
  session_count = Counter(game_session[column])
  for session_title in session_count.keys():
    label = session_title
    if column == 'title':
      label = event_data["dict_of_all_labels"][session_title]
    elif column == 'world':
      label = event_data["dict_of_all_world_labels"][session_title]
    count_dict[label] += session_count[session_title]
  return count_dict

#helper functions for games
def count_miss(df):
    total_miss = 0
    for e in range(len(df)):
        event_data = df['event_data'].iloc[e]
        number_of_misses = json.loads(event_data)['misses']
        total_miss += number_of_misses
    return total_miss

def count_num_correct(df):
  total_correct = 0
  total_wrong = 0

  for i in range(len(df)):
    json_event_data = json.loads(df['event_data'].iloc[i])
    if 'correct' in json_event_data:
      games_with_key = json_event_data
      if games_with_key['correct']:
        total_correct += 1
      else:
        total_wrong += 1
  return total_correct, total_wrong

In [0]:
# helper functions for assessments
# event_code 4020 indicates if the action taken was correct or incorrect
# this function returns the average 4020 accuracy attained in the assessment
def assessment_4020_acc(df, counter_dict, event_data):
    for assessment in ['Cauldron Filler (Assessment)', 'Bird Measurer (Assessment)',
              'Mushroom Sorter (Assessment)', 'Chest Sorter (Assessment)']:
        Assess_4020 = df[(df.event_code == 4020) & (df.title == event_data["dict_of_all_titles"][assessment])] # returns a df of event_code 4020 pertaining to above assessments

        true_attempts = 0
        false_attempts = 0  

        for i in range(len(Assess_4020)):
          if json.loads(Assess_4020.iloc[0]['event_data'])['correct']:  
            true_attempts += 1
          else:
            false_attempts += 1
            
        accuracy_4020 = true_attempts / (true_attempts + false_attempts) if (true_attempts + false_attempts) != 0 else 0
        counter_dict[assessment + "_4020_accuracy"] += (counter_dict[assessment + "_4020_accuracy"] + accuracy_4020) / 2.0

    return counter_dict

In [0]:
def reduce_train_test(train,test,event_data):
  train_compile = []
  for installation_id, user_data in tqdm(train.groupby('installation_id')):
    train_compile += get_data_for_each_user(user_data, event_data, train_labels)
  reduce_train = pd.DataFrame(train_compile)

  test_compile = []
  for installation_id, user_data in tqdm(test.groupby('installation_id')):
    test_compile.append(get_data_for_each_user(user_data, event_data, train_labels, True))
  reduce_test = pd.DataFrame(test_compile)


  return reduce_train, reduce_test
  # return reduce_test

In [0]:
train, test, event_data = encode_titles(train,test,train_labels)

## importing of event data file which was previously saved after doing feature engineering

In [0]:
import pickle
pickle_out = open("/content/drive/My Drive/introml/ted testing/event_data.pkl","wb")
pickle.dump(event_data, pickle_out)
pickle_out.close()

In [0]:
reduce_train, reduce_test = reduce_train_test(train, test, event_data)
# reduce_test = reduce_train_test(train, test, event_data)

100%|██████████| 1000/1000 [01:23<00:00, 12.55it/s]


In [0]:
reduce_train

,Clip,Activity,Assessment,Game,session_title,Magma Peak - Level 1,Sandcastle Builder (Activity),Slop Problem,Scrub-A-Dub,Tree Top City - Level 1,Ordering Spheres,All Star Sorting,Costume Box,Fireworks (Activity),12 Monkeys,Tree Top City - Level 2,Flower Waterer (Activity),Pirate's Tale,Mushroom Sorter (Assessment),Air Show,Treasure Map,Tree Top City - Level 3,Crystals Rule,Rulers,Bug Measurer (Activity),Bird Measurer (Assessment),Watering Hole (Activity),Magma Peak - Level 2,Dino Drink,Bubble Bath,Bottle Filler (Activity),Dino Dive,Crystal Caves - Level 1,Chow Time,Cauldron Filler (Assessment),Balancing Act,Crystal Caves - Level 2,Crystal Caves - Level 3,Chicken Balancer (Activity),Lifting Heavy Things,Pan Balance,Honey Cake,Happy Camel,Cart Balancer (Assessment),"Heavy, Heavier, Heaviest",Egg Dropper (Activity),Chest Sorter (Assessment),Leaf Leader,0,1,2,3,27253bdc,77261ab5,b2dba42b,1325467d,1bb5fbdb,5e812b27,9ee1c98c,84538528,37937459,6d90d394,7040c096,5a848010,c1cac9a2,26fd2d99,dcaede90,f71c4741,f7e47413,5c3d2b2f,08fd73f3,73757a5e,2b9272f4,37c53127,d88e8f25,ac92046e,cf82af56,4a09ace1,b7dc8128,4b5efe37,2c4e6db0,363d3849,587b5989,9e4c8c7b,1cc7cfca,2dc29e21,6043a2b4,c277e121,b120f2ac,d45ed6a1,ca11f653,daac11b0,1f19558b,d02b7a8e,4901243f,beb0a7b9,02a42007,e694a35b,b88f38da,884228c8,9b01374f,56cd3b43,a44b10dc,bbfe0445,5d042115,de26c3a6,598f4598,fcfdffb6,3bfd1a65,db02c830,a1e4395d,a52b92d5,28ed704e,9d29771f,c74f40cd,7da34a02,83c6c409,3dfd4aa4,fbaf3456,5f0eb72c,c7128948,25fa8af4,6c930e6e,a5be6304,15ba1109,65abac75,bcceccc6,06372577,f28c589a,a1bbe385,d88ca108,dcb55a27,1575e76c,28f975ea,14de4c5d,9b4001e4,58a0de5c,f5b8c21a,7423acbc,e04fb33d,48349b14,cc5087a3,5154fc30,3babcb9b,7cf1bc53,5e3ea25a,86c924c4,3ddc79c3,e720d930,3323d7e9,44cb4907,8b757ab8,c7f7f0e1,0a08139c,e79f3763,71fe8f75,363c86c9,022b4259,565a3990,f56e0afc,ec138c1c,1375ccb7,bdf49a58,51102b85,4a4c3d21,17113b36,ad2fc29c,e37a2b78,a16a373e,c58186bf,e64e2cfd,49ed92e9,bd701df8,f50fc6c1,d2e9262e,2fb91ec1,c952eb01,a6d66e51,71e712d8,51311d7a,5be391b5,c6971acf,a29c5338,6c517a88,7f0836bf,4d6737eb,f806dc10,792530f8,1996c610,74e5f8a7,e5734469,89aace00,77ead60d,4d911100,16dffff1,1cf54632,8d84fa81,99abe2bb,99ea62f3,0413e89d,15eb4a7d,1beb320a,8f094001,a0faea5d,c54cf6c5,3bb91dda,857f21c0,6f4adc4b,55115cbd,90ea0bac,5859dfb6,d06f75b5,895865f3,ecc36b7f,1340b8d7,d2278a3b,b7530680,67439901,bb3e370b,df4940d3,90efca10,d3f1e122,e9c52111,15a43e5b,160654fd,88d4a5be,d2659ab4,45d01abe,7525289a,f6947f54,8fee50e2,070a5291,3393b68b,9ed8f6da,ad148f58,85de926c,29bdd9ba,87d743c1,832735e1,76babcde,ab3136ba,7d5c30a2,7d093bf9,f93fc684,7ec0c298,0d1da71f,63f13dd7,7372e1a5,cfbd47c8,d185d3ea,4ef8cdd3,0330ab6a,2230fab4,56817e2b,47026d5f,cb6010f8,e3ff61fb,709b1251,6088b756,7961e599,c0415e5c,9de5e594,28a4eb9a,00c73085,90d848e0,532a2afb,923afab1,3ee399c3,37ee8496,30614231,2dcad279,392e14df,28520915,d3268efa,b5053438,756e5507,ea321fb1,84b0e0c8,4bb2f698,56bcd38d,499edb7c,cdd22e43,46cd75b4,85d1b0de,9c5ef70c,0086365d,a592d54e,6cf7d25c,a76029ee,d38c2fd7,731c0cbe,8d748b58,47efca07,5f5b2617,0d18d96c,d9c005dd,abc5811c,d51b1749,3bb91ced,c2baf0bd,8d7e386c,3d8c61b0,6bf9e3e1,69fdac0a,8af75982,1af8be29,3bf1cf26,a7640a16,36fa3ebe,c7fe2a55,a8a78786,7ad3efc6,65a38bf7,795e4a37,5e109ec3,5c2f29ca,828e68f9,d122731b,a8876db3,ecaab346,b74258a0,b2e5b0f1,f3cd5473,15f99afc,2a444e03,804ee27f,a5e9da97,907a054b,c51d8688,e7561dd2,bc8f2793,9b23e8ee,7ab78247,736f9581,b80e5e84,4c2ec19f,5b49460a,155f62a4,3dcdda7f,3ccd3f02,bd612267,a8efe47b,562cec5f,3d0b9317,0db6d71d,9ce586dd,93b353f2,ea296733,df4fe8b6,f54238ee,9e6b7fb5,b1d5101d,a1192f43,47f43a44,6f4bd64e,c189aaf2,37db1c2f,461eace6,9e34ea74,8ac7cce4,fd20ea40,29f54413,33505eae,7dfe6d8a,2a512369,86ba578b,f32856e4,262136f4,b012cd7f,3afde5dd,e5c9df6f,763fc34e,e57dd7af,67aa2ada,cb1178ad,e7e44842,46b50ba8,a2df0760,6aeafed4,e080a381,1c178d24,250513af,cf7638f3,9d4e7b25,acf5c23f,91561152,2b058fe3,3d63345e,53c6e11a,92687c59,5de79a6a,31973d56,4e5fc6f5,6f8106d9,04df9b66,5290eab1,3edf6747,3b2048ee,5348fd84,3afb49e

# Saving down of engineered features into csv files or pickle file

In [0]:
reduce_train.to_csv('/content/drive/My Drive/introml/ted testing/reduce_train.csv')
reduce_test.to_csv('/content/drive/My Drive/introml/ted testing/reduce_test.csv')

In [0]:
reduce_test.to_pickle("/content/drive/My Drive/introml/ted testing/reduce_test.pkl")
reduce_train.to_pickle("/content/drive/My Drive/introml/ted testing/reduce_train.pkl")